# Examples

## Example 1

Calculation of heights and flow rates in a pipe network. Extracted from:

- Brebbia, C. A., & Ferrante, A. J. (2013). Computational hydraulics. Butterworth-Heinemann.
- Example 4.2

The pipe system is shown in the figure. Each element (line) is assigned a number, as well as each node (junction point). The elements represent pipes, with their respective diameter and length. Some nodes have a known height $H$ and others have a demand for flow rate, which corresponds to an inlet or outlet of fluid.

<img src = "images/example4_2.svg" width = "350"> 

- $L_0 = 1000 \, m$, $D_0 = 0.4 \, m$
- $L_1 = 1000 \, m$, $D_1 = 0.2 \, m$
- $L_2 = 2000 \, m$, $D_2 = 0.283 \, m$
- $L_3 = 2000 \, m$, $D_3 = 0.283 \, m$
- $L_4 = 2000 \, m$, $D_4 = 0.573 \, m$
- $C_2 = 10 \, m^3 s^{-1}$
- $C_1 = 10 \, m^3 s^{-1}$
- $H_3 = 10 \, m$

The matrix of each element is obtained through the relationship of laminar flow between flow rate and heights:

$$\begin{bmatrix}Q_{k}^{i}\\Q_{j}^{i}\end{bmatrix}
=k^{i}\begin{bmatrix}1 & -1 \\-1 & 1\end{bmatrix}
\begin{bmatrix}H_{k}^{i}\\H_{j}^{i}\end{bmatrix}$$

where:

$$ k^i = \frac{\pi \rho g D_i^4}{128 L_i \mu} = \frac{\pi  g D_i^4}{128 L_i \nu}$$

First, the numpy library and the NetSimulation class are imported:

In [1]:
import numpy as np
import netsimulation as ns

An object of the NetSimulation class named mynet is created:

In [2]:
mynet = ns.NetSimulation()

Once the object is defined, the attributes of the object must be set to implement the problem. Starting with setting the number of nodes and elements:

In [3]:
mynet.n_elements = 5
mynet.n_nodes = 4

The connectivity matrix is established using a numpy array. Each row corresponds to an element, the first and second columns contain the start and end nodes of the element:

In [4]:
mynet.connectivity = np.array([[0, 1], 
                             [0, 2],
                             [1, 2],
                             [1, 3],
                             [2, 3]])

The boundary conditions for flow rates $\mathbf{b}$ and heights $\mathbf{x}$ are established using two vectors. One vector contains the nodes where the values are known, and another vector contains the respective values for each node:

In [5]:
mynet.nodes_x_known = np.array([0, 3])
mynet.values_x_known = np.array([20, 10])
mynet.nodes_b_known = np.array([1, 2])
mynet.values_b_known = np.array([10, 10])

The values of $k_i$ are stored in a vector $\mathbf{k}$ corresponding to each element. Below is the example, where the vector $\mathbf{k}$ is passed to the attribute of the class `k_element`:

In [6]:
mynet.length = np.array([1000, 1000, 2000, 2000, 2000])
diam = np.array([0.4, 0.2, 0.283, 0.283, 0.573])
nu = 1e-6
g = 9.81
mynet.k_element = np.pi * g * diam[:]**4 / (128. * nu * mynet.length[:])

The problem is now defined. You can select the solver using the `solver` attribute. Afterwards, use the `matrix_assembly` method of the class to assemble the matrix $\mathbf{A}$, then use the `solve` method to solve and calculate the unknowns.

In [7]:
mynet.solver = "cg"
mynet.matrix_assembly()
mynet.solve()

Access the results using the attributes of the class:

- `x` corresponds to the vector of unknowns $\mathbf{x}$, for this problem it corresponds to the heights.
- `b` is the vector of constants $\mathbf{b}$, in this case it corresponds to the flow rates.
- `dx` is the difference in heights $\Delta \mathbf{x}^i$ of the element.
- `Q` corresponds to the flow rate of the element, calculated as $\mathbf{Q}^i = \mathbf{k}^i \Delta \mathbf{x}^i$

In [8]:
print(mynet.x)
print(mynet.b)
print(mynet.Q)
print(mynet.dx)

[20.         19.44361741 11.49589496 10.        ]
[  6.70553639  10.          10.         -26.70553639]
[ 3.42943368  3.27610271  6.13715863  7.29227505 19.41326134]
[0.55638259 8.50410504 7.94772245 9.44361741 1.49589496]
